In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Drapery Length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Rugs & Mats B"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r'(\s?\d*\s?\-?\s?Inch\s?Width\s?x\s?\d*\s?\-?\s?Inch\s?L?o?n?g?)|(\s?\d*\s?\-?\s?Inch\s?x\s?\d*\s?\-?\s?Inch)|(\s?\d*\s?\-?\s?Inch)()'
# regex_pattern_x = r'(\s?\d*\s?\-?\s?Inch\s?Width\s?x\s?\d*\s?\-?\s?Inch\s?L?o?n?g?)|(\s?\d*\s?\-?\s?Inch\s?x\s?\d*\s?\-?\s?Inch)|(\s?\d*\s?\-?\s?Inch)()'


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
df = query_from_file(file_name='query/uncurated.sql', params=params)
df.head(1)

product_id external_id  \
0   817351954    67252510   

                                                                product_name  \
0  Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
0  Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   

   customer_id attributes   buckets values  
0            5       None  Curtains   None

In [5]:
df['buckets'].explode().value_counts()

Curtains                       958
Dividers/Partitions/Screens     56
Curtains Variety Packs           5
Name: buckets, dtype: int64

In [6]:
metric='in'

range_string= """
 101 in - 105 in
106 in - 110 in
111 in - 115 in
116 in - 120 in
121 in - 125 in
126 in - 130 in
131 in - 135 in
136 in - 140 in
20 in - 25 in
26 in - 30 in
31 in - 35 in
36 in - 40 in
41 in - 45 in
46 in - 50 in
51 in - 55 in
56 in - 60 in
61 in - 65 in
66 in - 70 in
71 in - 75 in
76 in - 80 in
81 in - 85 in
86 in - 90 in
91 in - 95 in
96 in - 100 in
Above 140 in
Under 20 in 
"""

In [7]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df['product_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern, x))
df

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1014   817181049    60837714   
1015   817471422    60836328   
1016   988569378    45577727   
1017   988548047    42382300   
1018   817178009    60061072   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1015                                   Sweet Jojo Designs Navy and Orange Stripe Window Valance   
1016                                    Disney® Minnie's Petal Perfect Appliqued Window Valance   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   
1018                                     Sweet Jojo Designs Solid Navy Window Curtain Panel Set   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       long_desc  \
0                                                                                                                                                                                                                                                                                     Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                                                                                                                      Grace your wi

In [9]:
df['product_matches'].explode().value_counts()

 84-Inch                           242
 95-Inch                           138
 63-Inch                           112
 108-Inch                          104
 96-Inch                            23
 54-Inch                            12
 24-Inch                             8
 36-Inch                             6
 72-Inch x 96-Inch                   5
 72-Inch                             5
 72-Inch x 86-Inch                   5
 40-Inch                             4
 60-Inch                             4
 54-Inch x 80-Inch                   4
 54-Inch x 78-Inch                   3
 45-Inch                             3
75-Inch x 72-Inch                    3
 120-Inch                            3
 39-Inch x 64-Inch                   2
63-Inch x 72-Inch                    2
 132-Inch                            2
 108-Inch Width x 120-Inch Long      2
 90-Inch                             2
 72-Inch x 72-Inch                   2
 108-Inch Width x 95-Inch Long       1
 35-Inch x 64-Inch       

In [10]:
df=df[(df['product_matches'].astype(str)!='[]')]
df

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1004   817682068    63633740   
1005   817672864    64924069   
1008   817187460    62408417   
1014   817181049    60837714   
1017   988548047    42382300   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1004                         Textured 2-Pack 108-Inch Grommet Top Window Curtain Panels in Navy   
1005                           Roxy Printed 95-Inch Rod Pocket Window Curtain Panel in Amethyst   
1008                 The Peanutshell™ Lattice 84-Inch Blackout Window Panels in Gold (Set of 2)   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0                                                                                                                                                                                           Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                               Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                            Grace your windows with this ethereal Window Panel from Valeron's Estate Collection. Crafted in luxurious silk, the panel features an opulent sheen that instantly transforms your home into a chic haven. Measures 50\" wide. Featuring a sumptuous, luxurious sheen the Valeron Lustre Solid Window Curtain Panel brings a touch of restrained elegance to any room Single-width pa

In [11]:
error

NameError: name 'error' is not defined

In [12]:
def split_col(data):
    data['product_matches']=data['product_matches'].str.get(0)
#     data=data.dropna()
    data['product_matches']=data['product_matches'].apply(lambda x:x.lower())
#     data[['length', 'width']]=data['product_matches'].str.split("x",expand=True,)

In [13]:
split_col(df) #might throw an error and have to drop nan
df

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1004   817682068    63633740   
1005   817672864    64924069   
1008   817187460    62408417   
1014   817181049    60837714   
1017   988548047    42382300   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1004                         Textured 2-Pack 108-Inch Grommet Top Window Curtain Panels in Navy   
1005                           Roxy Printed 95-Inch Rod Pocket Window Curtain Panel in Amethyst   
1008                 The Peanutshell™ Lattice 84-Inch Blackout Window Panels in Gold (Set of 2)   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0                                                                                                                                                                                           Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                               Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                            Grace your windows with this ethereal Window Panel from Valeron's Estate Collection. Crafted in luxurious silk, the panel features an opulent sheen that instantly transforms your home into a chic haven. Measures 50\" wide. Featuring a sumptuous, luxurious sheen the Valeron Lustre Solid Window Curtain Panel brings a touch of restrained elegance to any room Single-width pa

In [14]:
df[['length', 'width']]=df['product_matches'].str.split("x",expand=True,)

In [15]:
df

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1004   817682068    63633740   
1005   817672864    64924069   
1008   817187460    62408417   
1014   817181049    60837714   
1017   988548047    42382300   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1004                         Textured 2-Pack 108-Inch Grommet Top Window Curtain Panels in Navy   
1005                           Roxy Printed 95-Inch Rod Pocket Window Curtain Panel in Amethyst   
1008                 The Peanutshell™ Lattice 84-Inch Blackout Window Panels in Gold (Set of 2)   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0                                                                                                                                                                                           Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                               Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                            Grace your windows with this ethereal Window Panel from Valeron's Estate Collection. Crafted in luxurious silk, the panel features an opulent sheen that instantly transforms your home into a chic haven. Measures 50\" wide. Featuring a sumptuous, luxurious sheen the Valeron Lustre Solid Window Curtain Panel brings a touch of restrained elegance to any room Single-width pa

In [16]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)

In [17]:
drop_characters(df)
df

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1004   817682068    63633740   
1005   817672864    64924069   
1008   817187460    62408417   
1014   817181049    60837714   
1017   988548047    42382300   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1004                         Textured 2-Pack 108-Inch Grommet Top Window Curtain Panels in Navy   
1005                           Roxy Printed 95-Inch Rod Pocket Window Curtain Panel in Amethyst   
1008                 The Peanutshell™ Lattice 84-Inch Blackout Window Panels in Gold (Set of 2)   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0                                                                                                                                                                                           Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                               Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                            Grace your windows with this ethereal Window Panel from Valeron's Estate Collection. Crafted in luxurious silk, the panel features an opulent sheen that instantly transforms your home into a chic haven. Measures 50\" wide. Featuring a sumptuous, luxurious sheen the Valeron Lustre Solid Window Curtain Panel brings a touch of restrained elegance to any room Single-width pa

In [18]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")


In [19]:
def min_max(data):
    data['length']=data['length'].astype(int)
    data['width']=data['width'].astype(int)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [20]:
min_max(df) #error base 10 when there is only a value for max and not min
df

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1004   817682068    63633740   
1005   817672864    64924069   
1008   817187460    62408417   
1014   817181049    60837714   
1017   988548047    42382300   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1004                         Textured 2-Pack 108-Inch Grommet Top Window Curtain Panels in Navy   
1005                           Roxy Printed 95-Inch Rod Pocket Window Curtain Panel in Amethyst   
1008                 The Peanutshell™ Lattice 84-Inch Blackout Window Panels in Gold (Set of 2)   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0                                                                                                                                                                                           Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                               Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                            Grace your windows with this ethereal Window Panel from Valeron's Estate Collection. Crafted in luxurious silk, the panel features an opulent sheen that instantly transforms your home into a chic haven. Measures 50\" wide. Featuring a sumptuous, luxurious sheen the Valeron Lustre Solid Window Curtain Panel brings a touch of restrained elegance to any room Single-width pa

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [21]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  101  and num <= 105 :
        return " 101  - 105 in"
        
    if num >= 106  and num <= 110 :
        return "106  - 110 in"
        
    if num >= 111  and num <= 115 :
        return "111  - 115 in"
        
    if num >= 116  and num <= 120 :
        return "116  - 120 in"
        
    if num >= 121  and num <= 125 :
        return "121  - 125 in"
        
    if num >= 126  and num <= 130 :
        return "126  - 130 in"
        
    if num >= 131  and num <= 135 :
        return "131  - 135 in"
        
    if num >= 136  and num <= 140 :
        return "136  - 140 in"
        
    if num >= 20  and num <= 25 :
        return "20  - 25 in"
        
    if num >= 26  and num <= 30 :
        return "26  - 30 in"
        
    if num >= 31  and num <= 35 :
        return "31  - 35 in"
        
    if num >= 36  and num <= 40 :
        return "36  - 40 in"
        
    if num >= 41  and num <= 45 :
        return "41  - 45 in"
        
    if n

In [22]:
matches=df
matches

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1004   817682068    63633740   
1005   817672864    64924069   
1008   817187460    62408417   
1014   817181049    60837714   
1017   988548047    42382300   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1004                         Textured 2-Pack 108-Inch Grommet Top Window Curtain Panels in Navy   
1005                           Roxy Printed 95-Inch Rod Pocket Window Curtain Panel in Amethyst   
1008                 The Peanutshell™ Lattice 84-Inch Blackout Window Panels in Gold (Set of 2)   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0                                                                                                                                                                                           Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                               Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                            Grace your windows with this ethereal Window Panel from Valeron's Estate Collection. Crafted in luxurious silk, the panel features an opulent sheen that instantly transforms your home into a chic haven. Measures 50\" wide. Featuring a sumptuous, luxurious sheen the Valeron Lustre Solid Window Curtain Panel brings a touch of restrained elegance to any room Single-width pa

In [23]:
matches[curation_col] = matches['Max'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

product_id external_id  \
0      817351954    67252510   
1     1212988376    69671168   
2     1213085253    43020669   
3         485159    43888092   
4     1213095242    45668081   
...          ...         ...   
1004   817682068    63633740   
1005   817672864    64924069   
1008   817187460    62408417   
1014   817181049    60837714   
1017   988548047    42382300   

                                                                                   product_name  \
0                     Peri Home Dayna Print 84-Inch Rod Pocket Window Curtain Panel in Amethyst   
1                          Simply Essential™ Engineered Floral 72-Inch x 86-Inch Shower Curtain   
2                                      Valeron Estate 84-Inch Silk Window Curtain Panel in Blue   
3     Valeron Lustre Grommet Top 108-Inch Width x 95-Inch Long Window Curtain Panel in Charcoal   
4             Valeron Belvedere 95-Inch Grommet Top Room-Darkening Window Curtain Panel in Pink   
...                                                                                         ...   
1004                         Textured 2-Pack 108-Inch Grommet Top Window Curtain Panels in Navy   
1005                           Roxy Printed 95-Inch Rod Pocket Window Curtain Panel in Amethyst   
1008                 The Peanutshell™ Lattice 84-Inch Blackout Window Panels in Gold (Set of 2)   
1014                 Sweet Jojo Designs Navy and Orange Stripe 84-Inch Window Panels (Set of 2)   
1017                    Tadpoles Layered Tulle 63-Inch Rod Pocket Window Curtain Panel in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0                                                                                                                                                                                           Add serenity to your decor with the Peri Home Dayna Rod Pocket Window Curtain Panel. This beautiful panel is designed in a chic botanical floral medallion print styled in a repeating watercolor motif. Hangs on a rod pocket. A botanical floral medallion displayed in a repeating watercolor design 100% polyester Machine wash Panel hangs on a rod pocket Panels are sold individually and measure 50\" W Rod pocket measures 3\" diameter Fits a rod up to 1.25\" diameter Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                               Why we love this... Machine washable for easy care Elegant floral design Fresh and clean botanical design 12 shower hook holes Certified STANDARD 100 by OEKO-TEX® 100% polyester microfiber Machine wash Shower hooks and liner not included Imported   
2                                                            Grace your windows with this ethereal Window Panel from Valeron's Estate Collection. Crafted in luxurious silk, the panel features an opulent sheen that instantly transforms your home into a chic haven. Measures 50\" wide. Featuring a sumptuous, luxurious sheen the Valeron Lustre Solid Window Curtain Panel brings a touch of restrained elegance to any room Single-width pa

In [24]:
matches['Q:drapery_length'].explode().value_counts()

81  - 85 in       243
91  - 95 in       138
61  - 65 in       115
106  - 110 in     106
96  - 100 in       27
71  - 75 in        15
51  - 55 in        12
36  - 40 in        10
20  - 25 in         8
86  - 90 in         8
76  - 80 in         7
116  - 120 in       5
56  - 60 in         5
n/a                 5
41  - 45 in         4
131  - 135 in       2
 101  - 105 in      1
66  - 70 in         1
Name: Q:drapery_length, dtype: int64

In [25]:
matches[matches['Q:drapery_length'].astype(str)=='n/a']

product_id external_id  \
93    817530808    62412445   
594  1207678724    69704999   
637   817268624    62779630   
666   817418708    69522944   
968   817187462    62408431   

                                                                            product_name  \
93                            Marquee 144-Inch Rod Pocket Window Curtain Panel in Bronze   
594  Sun Zero Bella Room Darkening 18-Inch Rod Pocket Window Curtain Valance in Wine Red   
637        Madison Park Harper Solid Crushed Sheer 216-Inch Scarf Window Valance in Grey   
666                                       No.918® Delia 17-Inch Curtain Valance in White   
968                               The Peanutshell™ 14-Inch Window Valance in Light Coral   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
93                                                                                                                                                                                                                              Add a designer touch to your window with the Marquee Pinch Pleat Back Tab Curtain Panel. The lined panel offers better privacy and features a traditional smooth textured finish with just the right amount of color for an extra-stylish look. Traditional smooth textured finish with just the right amount of color 100% polyester Machine wash and dry Pinch pleat with back tabs Panels are sold individually and measure 30” W Rod pocket measures 1-1/4\" diameter Imported   
594  The Sun Zero Bella Room Darkening Rod Pocket Curtain Valance delicately adorns the top of any window in your home. Pair with Sun Zero curtains or hang alone for maximum light exposure. Room Darkening technology blocks out some unwanted light while enhancing privacy Energy efficient design reduces energy lost through your windows by up to 25% Thermal properties help keep summer heat and winter chill out of your home Noise reduction up to 25% helps you get a sounder, more uninterrupted sleep Each valance sold separately (image features 3 valances) Machine wash and dry Rod Pocket top treatment Panel sold individually and measures 54\" W Fits a rod up to 1.5\" in diameter Imported   
637                                                                                                                                                                                                                                                                                                      Add a touch of modern elegance to your room with the Madison Park Harper Solid Crushed Sheer Scarf Window Valance. The lightweight sheer fabric is beautifully embellished with crushed detailing for added texture to complement any room's décor. Lightweight sheer fabric with crushed detailing for a beautiful texture 100% polyester Machine wash Hangs on tab top Valance measures 42\" W Imported   
666                                                                                                                                                                                                                                                                                           Add a pop of pattern to your window with the Delia Embroidered Floral from No. 918®. Embroidered floral design Textured sheer fabric gently diffuses light Rod pocket fits curtain rods up to 1.5\" in diam

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [26]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [27]:
looks_good('Bed Bath & Beyond', attribute, df, matches)